# Clustering Crypto

In [48]:
# Initial imports
import requests
import pandas as pd
from path import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [49]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
#url = "https://min-api.cryptocompare.com/data/all/coinlist&api_key=f38c7de2ccd3ab16371f43d09177bb0a5d7096c39ef588e128035431a82590c1"
#url = "https://min-api.cryptocompare.com/stats/rate/limit"

In [50]:
raw = requests.get(url)


In [51]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
data = raw.json()
coins = pd.DataFrame(data["Data"])
transposed = coins.transpose()
transposed.columns


Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'BlockNumber', 'NetHashesPerSecond',
       'BlockReward', 'BlockTime', 'AssetLaunchDate', 'MaxSupply',
       'MktCapPenalty', 'IsUsedInDefi', 'IsUsedInNft', 'PlatformType',
       'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty'],
      dtype='object')

In [52]:
coins_df = transposed[["CoinName", "Algorithm", "IsTrading", "ProofType", "TotalCoinsMined", "MaxSupply"]]
coins_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


# Unable to find "TotalCoinSupply" on API.

In [53]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [54]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df = df.drop(columns=["Unnamed: 0"])
trading = df[df["IsTrading"] == True]
trading.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [55]:
# Keep only cryptocurrencies with a working algorithm
trading["Algorithm"].isnull().sum()

0

In [56]:
trading["Algorithm"].isna().sum()

0

In [57]:
# Remove the 'IsTrading' column
coins = trading.drop(columns=["IsTrading"])
coins.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [58]:
# Remove rows with at least 1 null value
coins.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [59]:
# Remove rows with cryptocurrencies having no coins mined
# Drop rows where there are 'N/A' text values
dropped = coins.dropna()
dropped.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [60]:
final = dropped[dropped["TotalCoinSupply"] != 0]
final.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [61]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coin_name = dropped["CoinName"]
coin_name.head()

0      42 Coin
2      404Coin
4          808
5    EliteCoin
7      Bitcoin
Name: CoinName, dtype: object

In [62]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
clean = dropped.drop(columns=["CoinName"])
clean.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
4,SHA-256,PoW/PoS,0.000000e+00,0
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000


In [63]:
clean["TotalCoinSupply"].astype(float)
final = clean[clean.TotalCoinSupply != 0]
final.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
4,SHA-256,PoW/PoS,0.000000e+00,0
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000


In [64]:
# Create dummy variables for text features
final_df = pd.get_dummies(final, columns=["Algorithm", "ProofType"])
final_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# Standardize data
stand = StandardScaler().fit_transform(final_df)

### Reducing Dimensions Using PCA

In [66]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(stand)

In [67]:
# Create a DataFrame with the principal components data
crypto_df = pd.DataFrame(data=crypto_pca, columns=["PC1", "PC2", "PC3"])
crypto_df.head()

,PC1,PC2,PC3
0,-0.261384,1.181964,-0.476052
1,-0.247180,1.180321,-0.476300
2,-0.173099,0.817465,-0.293196
3,0.189760,1.924834,-0.468079
4,-0.158740,-1.424609,0.137113


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [68]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [69]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0
              )
# Fit the model
model.fit(crypto_df)

# Predict clusters
predictions = model.predict(crypto_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_df["predictions"] = model.labels_
crypto_df.head()


,PC1,PC2,PC3,predictions
0,-0.261384,1.181964,-0.476052,0
1,-0.247180,1.180321,-0.476300,0
2,-0.173099,0.817465,-0.293196,0
3,0.189760,1.924834,-0.468079,0
4,-0.158740,-1.424609,0.137113,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [70]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="predictions",
    symbol="predictions",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [73]:
# Table with tradable cryptos
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path)
df.head()
df = df.drop(columns=["Unnamed: 0"])
trading = df[df["IsTrading"] == True]
trading.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [78]:
# Print the total number of tradable cryptocurrencies
trading["IsTrading"].sum()

1144

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot



In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
final_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
)